## Julia language demo

### Tony Kelman, Mar 2nd 2016, eMLDSC

https://github.com/tkelman/eMLDSC

http://julialang.org, https://juliabox.org, http://juliacomputing.com

## 2-language problem: performance OR productivity

- C, C++, or Java for performance
- Python, R, or Matlab for productivity

Julia: new (but familiar) language, best of both worlds

![Performance relative to C](benchmarks_time.png)
Microbenchmark performance, time relative to C

![Performance and code size](benchmarks_codesize.png)

### Intro syntax example - Dual Numbers

Of the form $a + b ε$, similar to complex numbers but with $ε² = 0$ instead of $i² = -1$

In [1]:
# type => by-reference semantics,
# immutable => by-value semantics
immutable DualNumber{T}
    value::T
    epsilon::T
end
# type parameter {T} constrains typeof(value) == typeof(epsilon)

In [2]:
# import addition and multiplication from Base to extend them with new methods
import Base: +, *
# one-liner function definition:
+(x::DualNumber, y::DualNumber) = DualNumber(x.value + y.value, x.epsilon + y.epsilon)
# longer form function definition:
function *(x::DualNumber, y::DualNumber)
    return DualNumber(x.value * y.value, x.value * y.epsilon + y.value * x.epsilon)
end

* (generic function with 139 methods)

In [3]:
# compare LLVM and native code generated for DualNumber{Int64} vs DualNumber{Float64}
x_i64 = DualNumber(1, 2)

DualNumber{Int64}(1,2)

In [4]:
y_i64 = DualNumber(3, 4)

DualNumber{Int64}(3,4)

In [5]:
@code_llvm x_i64 * y_i64


define void @"julia_*_1478"(%DualNumber* sret, %DualNumber*, %DualNumber*) {
top:
  %3 = load %DualNumber* %1, align 8
  %4 = extractvalue %DualNumber %3, 0
  %5 = extractvalue %DualNumber %3, 1
  %6 = load %DualNumber* %2, align 8
  %7 = extractvalue %DualNumber %6, 0
  %8 = extractvalue %DualNumber %6, 1
  %9 = mul i64 %7, %4
  %10 = insertvalue %DualNumber undef, i64 %9, 0
  %11 = mul i64 %8, %4
  %12 = mul i64 %7, %5
  %13 = add i64 %11, %12
  %14 = insertvalue %DualNumber %10, i64 %13, 1
  store %DualNumber %14, %DualNumber* %0, align 8
  ret void
}


In [6]:
@code_native x_i64 * y_i64

	.text
Filename: In[2]
Source line: 7
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 7
	movq	(%rdx), %r9
	movq	8(%r8), %r10
Source line: 7
	imulq	%r9, %r10
Source line: 7
	movq	(%r8), %rax
	movq	8(%rdx), %rdx
Source line: 7
	imulq	%rax, %rdx
	imulq	%r9, %rax
	movq	%rax, (%rcx)
	addq	%r10, %rdx
	movq	%rdx, 8(%rcx)
	movq	%rcx, %rax
	popq	%rbp
	ret


In [7]:
x_f64 = DualNumber(1.0, 2.0)

DualNumber{Float64}(1.0,2.0)

In [8]:
y_f64 = DualNumber(3.0, 4.0)

DualNumber{Float64}(3.0,4.0)

In [9]:
@code_llvm x_f64 * y_f64


define void @"julia_*_1588"(%DualNumber.8* sret, %DualNumber.8*, %DualNumber.8*) {
top:
  %3 = load %DualNumber.8* %1, align 8
  %4 = extractvalue %DualNumber.8 %3, 0
  %5 = extractvalue %DualNumber.8 %3, 1
  %6 = load %DualNumber.8* %2, align 8
  %7 = extractvalue %DualNumber.8 %6, 0
  %8 = extractvalue %DualNumber.8 %6, 1
  %9 = fmul double %4, %7
  %10 = insertvalue %DualNumber.8 undef, double %9, 0
  %11 = fmul double %4, %8
  %12 = fmul double %5, %7
  %13 = fadd double %11, %12
  %14 = insertvalue %DualNumber.8 %10, double %13, 1
  store %DualNumber.8 %14, %DualNumber.8* %0, align 8
  ret void
}


In [10]:
@code_native x_f64 * y_f64

	.text
Filename: In[2]
Source line: 7
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 7
	movsd	(%rdx), %xmm1
	movsd	8(%r8), %xmm0
Source line: 7
	mulsd	%xmm1, %xmm0
Source line: 7
	movsd	(%r8), %xmm2
Source line: 7
	mulsd	%xmm2, %xmm1
	mulsd	8(%rdx), %xmm2
	movsd	%xmm1, (%rcx)
	addsd	%xmm0, %xmm2
	movsd	%xmm2, 8(%rcx)
	movq	%rcx, %rax
	popq	%rbp
	ret


In [14]:
+(a::DualNumber, b::Number) = DualNumber(a.value + b, a.epsilon)
#+(b::Number, a::DualNumber) = DualNumber(a.value + b, a.epsilon)
#1 + DualNumber(1.0, 2.0)

DualNumber{Float64}(2.0,2.0)

In [16]:
# custom printing for a type can be achieved by extending the `show` function:
function Base.show(io::IO, x::DualNumber)
    print(io, string(x.value, "+", x.epsilon, "ε"))
end
DualNumber(1, 2)

1+2ε

In [17]:
# To use DualNumber in linear algebra operations, just need to define `zero` value
Base.zero{T}(::Type{DualNumber{T}}) = DualNumber(zero(T), zero(T))

zero (generic function with 14 methods)

In [18]:
values1 = rand(1:10, 6, 4);
epsilon1 = rand(1:10, 6, 4);
dualmat1 = map(DualNumber, values1, epsilon1)

6x4 Array{DualNumber{Int64},2}:
 9+10ε  4+2ε  6+10ε  10+2ε
 4+2ε   3+1ε  9+6ε   8+1ε 
 5+10ε  5+3ε  7+3ε   4+6ε 
 6+5ε   4+2ε  8+1ε   4+4ε 
 8+6ε   2+6ε  1+5ε   1+6ε 
 1+4ε   8+8ε  1+2ε   6+5ε 

In [19]:
values2 = rand(1:10, 4, 3);
epsilon2 = rand(1:10, 4, 3);
dualmat2 = map(DualNumber, values2, epsilon2)

4x3 Array{DualNumber{Int64},2}:
 10+10ε  10+3ε  1+4ε
 5+9ε    8+2ε   8+4ε
 9+7ε    3+3ε   3+6ε
 6+1ε    4+7ε   7+4ε

In [20]:
# matrix multiplication
dualmat1 * dualmat2

6x3 Array{DualNumber{Int64},2}:
 224+390ε  180+277ε  129+198ε
 184+223ε  123+151ε  111+149ε
 162+326ε  127+231ε  94+183ε 
 176+249ε  132+163ε  90+156ε 
 105+277ε  103+185ε  34+161ε 
 95+223ε   101+194ε  110+175ε

In [21]:
# elementwise multiplication
dualmat1 .* dualmat1

6x4 Array{DualNumber{Int64},2}:
 81+180ε  16+16ε   36+120ε  100+40ε
 16+16ε   9+6ε     81+108ε  64+16ε 
 25+100ε  25+30ε   49+42ε   16+48ε 
 36+60ε   16+16ε   64+16ε   16+32ε 
 64+96ε   4+24ε    1+10ε    1+12ε  
 1+8ε     64+128ε  1+4ε     36+60ε 

In [23]:
# calling into C libraries
ccall(:sin, Float64, (Float64,), 1.0)

0.8414709848078965

In [24]:
buffer = Array(UInt8, 25)
ccall(:sprintf, Void, (Ptr{UInt8}, Ptr{UInt8}, Ptr{UInt8}), buffer, "%s\n", "Hello from C sprintf!")
len = ccall(:strlen, Cint, (Ptr{UInt8},), buffer)
bytestring(pointer(buffer), len)

"Hello from C sprintf!\n"

In [26]:
# calling into Python libraries - https://github.com/stevengj/PyCall.jl
Pkg.add("PyCall")
using PyCall
@pyimport scipy.optimize as so
# (see http://www.juliaopt.org for Julia optimization packages)
so.newton(x -> cos(x) - 3x, 1)

INFO: Nothing to be done


0.31675082877125116

In [27]:
# access, and modify, the syntax tree of code as a data structure
ex = :(cos(x) - 3x)
dump(ex)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol -
    2: Expr 
      head: Symbol call
      args: Array(Any,(2,))
        1: Symbol cos
        2: Symbol x
      typ: Any
    3: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol *
        2: Int64 3
        3: Symbol x
      typ: Any
  typ: Any


In [28]:
# macros begin with @, take expression structure of input,
# modify and return a different expression to execute
ex2 = copy(ex)
ex2.args[1] = :*
show(ex2)

:(cos(x) * (3x))

In [29]:
macro multiply(foo)
    foo.args[1] = :*
    return foo
end
@multiply cos(1) - 3

1.6209069176044193

In [30]:
cos(1) - 3

-2.4596976941318602

In [31]:
cos(1) * 3

1.6209069176044193